In [1]:
import os

folder_path = 'D:/antitrust/Data 1/data14'

# List all files in the folder
files = os.listdir(folder_path)

# Print the names of all documents
for file in files:
    print(file)

d1.PDF
d10.PDF
d11.PDF
d12.PDF
d13.PDF
d14.PDF
d15.PDF
d16.PDF
d17.PDF
d18.PDF
d19.PDF
d2.PDF
d3.PDF
d4.PDF
d5.PDF
d6.PDF
d7.PDF
d8.PDF
d9.PDF


to know how many cases are in this file

In [2]:
import fitz
import re

def extract_target_number(pdf_path):
    doc = fitz.open(pdf_path)

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for b in blocks:
            if "lines" in b:
                for line in b["lines"]:
                    for span in line["spans"]:
                        if (
                            'number' in b and b['number'] == 5
                            and 'size' in span and span['size'] == 12.0
                            and 'font' in span and span['font'] == 'Helvetica-Bold'
                            and page_num == 0  # Only on the first page
                        ):
                            match = re.search(r'\((\d+)\)', span["text"])
                            if match:
                                return int(match.group(1))

    doc.close()


In [3]:
print(extract_target_number('D:/antitrust/Data 1/data14/d2.PDF'))

1


get the title list

In [4]:
import fitz
import re

def extract_title(pdf_path):
    doc = fitz.open(pdf_path)
    extracted_text_list = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for b in blocks:
            if "lines" in b:
                for line in b["lines"]:
                    for span in line["spans"]:
                        if 'font' in span and span['font'] == 'Helvetica-Oblique':
                            result_string = re.sub(r',[^,]*$', '', span["text"].strip())
                            extracted_text_list.append(result_string)

        if len(extracted_text_list) >= extract_target_number(pdf_path):
            break

    doc.close()
    return extracted_text_list


In [5]:

pdf_path = 'D:/antitrust/Data 1/data14/d2.PDF'
# pdf_path = "D:/antitrust/verdict/Files (100).PDF"
result_list = extract_title(pdf_path)

In [7]:
result_list

['Aspen Skiing Co. v. Aspen Highlands Skiing Corp.']

get other information and write them into csv

In [17]:
import fitz
import csv

def getAndWrite(pdf_path, case_list):
    doc = fitz.open(pdf_path)
    matching_texts = []
    titles_list = set()

    title = None  # Variable to store 'number': 1 text
    court = None  # Variable to store 'number': 2 text
    date = None  # Variable to store 'number': 3 text
    no = None  # Variable to store 'number': 4 text
    consider_number_2 = False  # Flag to indicate whether to consider 'number': 2 text
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:
            if 'number' in b and b['number'] == 1:
                if "lines" in b:
                    # Reset text variables for each new 'number': 1
                    title = ""
                    court = None
                    date = None
                    no = None

                    for line in b["lines"]:
                        for span in line["spans"]:
                            if 'font' in span and span['font'] == 'Helvetica-BoldOblique':
                                title = span["text"].strip()
                                titles_list.add(title)
                                for case_name in case_list:
                                    if title.strip() == case_name.strip():
                                        consider_number_2 = True
                                        break  # Stop looking for 'number': 1 text once found
            if consider_number_2 and 'number' in b and b['number'] == 2:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                court = span["text"].strip()
            if consider_number_2 and 'number' in b and b['number'] == 3:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                date = date.strip() + span["text"].strip() if date else span["text"].strip()
            if consider_number_2 and 'number' in b and b['number'] == 4:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                no = span["text"].strip()
                                consider_number_2 = False
                                break  # Stop looking for 'number': 4 text once found
        # Check if 'number': 1, 'number': 2, 'number': 3, and 'number': 4 text are present, then append to the result
        if no:
            matching_texts.append({'pdf': pdf_path,'Title': title, 'Court': court, 'Date': date, 'No': no, 'page': page_num + 1 })

    doc.close()

    # Get the difference set 差集
    difference_set = set(case_list).symmetric_difference(titles_list)
    print(pdf_path)
    print(difference_set)
    print('======================================')
    return matching_texts
    


In [8]:
import fitz
import csv

def getAndWrite(pdf_path, case_list):
    doc = fitz.open(pdf_path)
    matching_texts = []
    titles_list = set()

    title = None  # Variable to store 'number': 1 text
    court = None  # Variable to store 'number': 2 text
    date = None  # Variable to store 'number': 3 text
    no = None  # Variable to store 'number': 4 text
    consider_number_2 = False  # Flag to indicate whether to consider 'number': 2 text
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:
            if 'number' in b and b['number'] == 2:
                if "lines" in b:
                    # Reset text variables for each new 'number': 1
                    title = ""
                    court = None
                    date = None
                    no = None

                    for line in b["lines"]:
                        for span in line["spans"]:
                            if 'font' in span and span['font'] == 'Helvetica-BoldOblique':
                                title = span["text"].strip()
                                titles_list.add(title)
                                for case_name in case_list:
                                    if title.strip() == case_name.strip():
                                        consider_number_2 = True
                                        break  # Stop looking for 'number': 1 text once found
            if consider_number_2 and 'number' in b and b['number'] == 3:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                court = span["text"].strip()
            if consider_number_2 and 'number' in b and b['number'] == 4:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                date = date.strip() + span["text"].strip() if date else span["text"].strip()
            if consider_number_2 and 'number' in b and b['number'] == 5:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                no = span["text"].strip()
                                consider_number_2 = False
                                break  # Stop looking for 'number': 4 text once found
        # Check if 'number': 1, 'number': 2, 'number': 3, and 'number': 4 text are present, then append to the result
        if no:
            matching_texts.append({'pdf': pdf_path,'Title': title, 'Court': court, 'Date': date, 'No': no, 'page': page_num + 1 })

    doc.close()

    # Get the difference set 差集
    difference_set = set(case_list).symmetric_difference(titles_list)
    print(pdf_path)
    print(difference_set)
    print('======================================')
    return matching_texts

In [9]:
# Initialize CSV file outside the files loop
output_csv = "output.csv"
with open(output_csv, 'w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['pdf', 'Title', 'Court', 'Date', 'No', 'page']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

for file in files:
    pdf_path = os.path.join(folder_path, file)
    case_list = extract_title(pdf_path)
    matching_texts = getAndWrite(pdf_path, case_list)
    
    # Append to the CSV file without overwriting it
    with open(output_csv, 'a', newline='', encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        # Write the data
        for result in matching_texts:
            writer.writerow(result)


D:/antitrust/Data 1/data14\d1.PDF
{'LEdHN[4]', '[1678]', 'antitrust law', '[1680]'}
D:/antitrust/Data 1/data14\d10.PDF
set()
D:/antitrust/Data 1/data14\d11.PDF
{'LEdHN[13]', 'antitrust law'}
D:/antitrust/Data 1/data14\d12.PDF
{'[***658]', 'LEdHN[17]', 'LEdHN[4]', 'LEdHN[5]', 'HN5'}
D:/antitrust/Data 1/data14\d13.PDF
{'LEdHN[12]', 'HN3', 'antitrust law'}
D:/antitrust/Data 1/data14\d14.PDF
{'LEdHN[10]', 'HN6', 'antitrust law'}
D:/antitrust/Data 1/data14\d15.PDF
{'ANTITRUST LAW', 'LEdHN15.'}
D:/antitrust/Data 1/data14\d16.PDF
{'[**710]', 'HN11', 'LEdHN[11]'}
D:/antitrust/Data 1/data14\d17.PDF
{'[****49]', 'HN2', 'Antitrust Law'}
D:/antitrust/Data 1/data14\d18.PDF
{'[*359]', '[**1739]', 'HN7', 'LEdHN[18]', '[***931]', 'LEdHN[13]', 'LEdHN[14]', 'LEdHN[6]', '[**1734]', 'LEdHN[36]', 'LEdHN[12]', 'antitrust law', 'LEdHN[25]'}
D:/antitrust/Data 1/data14\d19.PDF
{'LEdHN[1A]', 'LEdHN[11]', 'LEdHN[6]', 'HN13', 'Antitrust Law', 'LEdHN[1I]'}
D:/antitrust/Data 1/data14\d2.PDF
{'HN2'}
D:/antitrust/Dat

In [72]:
# Example usage:
pdf_path = "D:/antitrust/verdict/Federal Circuit (30).PDF"
# pdf_path = "D:/antitrust/verdict/Files (100).PDF"
case_list = result_list
output_csv = "output.csv"
getAndWrite(pdf_path, case_list, output_csv)

{'1', '9 to 5 Organization for Women Office Workers v. Board of Governors of', '9 to 5 Organization for Women Office Workers v. Board of Governors of Federal Reserve System'}


In [4]:
import os

# Set the path of the directory
folder_path = 'D:/antitrust/Data 1/data14'

# Get a list of files in the directory
files = os.listdir(folder_path)

# Initialize a counter for the file names
counter = 1

for file_name in files:
    # Get the file extension
    extension = os.path.splitext(file_name)[1]
    
    # Construct the new file name
    new_name = f"d{counter}{extension}"
    
    # Get the full file paths
    old_file_path = os.path.join(folder_path, file_name)
    new_file_path = os.path.join(folder_path, new_name)
    
    # Rename the file
    os.rename(old_file_path, new_file_path)
    
    # Increment the counter
    counter += 1

print("Files have been renamed.")


Files have been renamed.


In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('your_file.csv')

# Display the first few rows of the DataFrame
print(df.head())